# Project Management

## Internals

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import IPython.display as dis

### Param

In [69]:
_pathTasks='tasksProjet.csv'
_pathAvail='availReel.csv'
_ProjectStartDate = dt.datetime(year=2016,month=11,day=21)
_1DayTimeDelta = dt.timedelta(1)

### Load data

In [116]:
_TaskHeaders = ['TaskId','Subject','Name','WorkLoad','Who','StartDate','EndDate','Priority','PredecessorTaskId']

def LoadTask(path):
        data = pd.read_csv(path, header=0,names=_TaskHeaders, dtype={'PredecessorTaskId':object},delimiter=';',decimal=',')
        data.StartDate = pd.to_datetime(data.StartDate)
        data.EndDate = pd.to_datetime(data.EndDate)
        data.index=data.TaskId
        if len(data)!=len(data.TaskId.unique()):
            raise Exception ('Tasks Id is not unique')
        return data
def LoadAvailability(path):
    '''
    This function load an availability matrix
    '''
    availCSV = pd.read_csv(path)
    availCSV.Date = pd.to_datetime(availCSV.Date)
    availCSV.sort_values(inplace=True,by='Date')
    return availCSV
    

### Save

In [4]:
def SaveData(data,path):
    data.to_csv(path)

### Domain
* Find tasks of a personn
* Order tasks by priority then ID (just in case)
* Compute end dates
    * Find previous task of person and previous task id
    * Based on the max start date and workload, iterate to see the day when the task will be finised


In [5]:
def FindPreviousTaskOfSomeone(tasks, current):
    '''
    Find the previous task in order of doing
    Expects an order tasks pandas by priority desc and taskid 
    '''
    whosTasks = FindTasksOfSomeone(tasks,current.Who)    
    prevTasks= whosTasks.loc[:current.TaskId]
    if len(prevTasks)<2 :
        return None
    else :
        return prevTasks.iloc[-2]
    
    

In [6]:
def FindTasksOfSomeone(tasks, who):
    return tasks[tasks.Who==who]
def FindTaskById(tasks, taskid):
    return tasks.loc[int(taskid)]
def OrderTaskByPriority(tasks):
    return tasks.sort_values(by=['Priority','TaskId'], ascending=[False,True])

In [29]:
def InitEmptyOccupation(avail,startDate, endDate):
    users = avail.Who.unique()
    duration = (endDate-startDate).days
    return pd.DataFrame(data=np.zeros(shape=(duration,len(users)), dtype=np.int32),\
                        columns=users,\
                        index=np.arange(startDate,endDate, dtype='datetime64[D]'))

In [104]:
def ComputeEndDate(availability, task):
    '''
    Compute the end date based on the occupation of someone, a start date and a duration
    '''    
    availWho = availability[(availability.Who==task.Who) & (availability.Date>=task.StartDate)]  
    currentOccupation = 0
    endDate=None    
    for index,item in availWho.iterrows():
        currentOccupation=currentOccupation+item.Available
        #print 'Task %s with workload %s at Date %s & occupation %s' %(task.TaskId,task.WorkLoad, item.Date, currentOccupation)
        if(float(currentOccupation)>=float(task.WorkLoad)):
            endDate=item.Date
            break;
    if(endDate==None):
        raise Exception('Task cannot be ended',task)
    return endDate
        

In [9]:
def SetOccupiedBy(occupation,task):
    occupation.loc[task.StartDate:task.EndDate,task.Who]=task.TaskId

In [89]:
def OrderTasksByPriorityAndPredecessor(tasks):
    tasks = OrderTaskByPriority(tasks)
    indexTasks = tasks.TaskId.tolist()
    #print indexTasks
    '''
        While on the number of element
            If has predecessor
                Get its location 
                If bigger > cursor --> move after  (check if last item)          
            If moved : do not move cursor
            Else : next item
        '''
    lItems = len(indexTasks)
    cursor = 0
    moved = False 
    listOfMoved = []
    while (cursor<lItems):  
        taskI = tasks.loc[indexTasks[cursor]]
        moved=False
        #print '  Current task %s at position %s has predecessor %s'%(taskI.TaskId,cursor,taskI.PredecessorTaskId)
        if (pd.isnull(taskI.PredecessorTaskId)==False) :
            predecIdx = indexTasks.index(int(taskI.PredecessorTaskId))    
            #print '  Predecessor %s is at position %s' %(taskI.PredecessorTaskId,predecIdx)
            if(predecIdx>cursor):
                #print '  Task %s with position %s must be moved after task %s at position %s' \
                #    %(taskI.TaskId,cursor,taskI.PredecessorTaskId,predecIdx)
                #Check if this task has already been moved, if yes we have a cycle
                if(taskI.TaskId in listOfMoved):
                    raise Exception('There is a cycle between tasks found on ',taskI.TaskId)
                listOfMoved.append(taskI.TaskId)
                indexTasks.pop(cursor)
                #print '  Popped %s' %(indexTasks)
                Moved=True
                cursor -=1
                if(predecIdx==lItems-1):
                    indexTasks.append(taskI.TaskId)
                else:
                    indexTasks.insert(predecIdx,taskI.TaskId)
        if moved==False:
                cursor +=1
        #print '  %s' %(indexTasks)
    return tasks.loc[indexTasks]

In [11]:
def ComputeOccupation(avail,tasks):
    '''
    Computer the tasks start and end date based on 
    
    1. Order of computation is based on the priority and PredecessorTaskId
        First the algorithm order by priority then check the predecessors 
    and moves each row just after its predecessor if it is before
        The algorithm will however detect incorrect cases :
    When it parcours the tasks, it checks if the PredecessorTaskId task has a end date. 
            If not, sends an error. 
            If yes uses the maximum between the previous task of the person and the predecessor
    2. By personn iterate over its tasks and compute the dates 
        2.1 If first task : prevStartDate=_projectStartDate
            else prevStartDate=previous.EndDate+1 (we cheat and say one task per day!)
        2.2 Check the predecessor end date : 
            If predecessor.EndDate = NaT --> exception (cycle)
        2.3 Take the max between both
        2.4 Compute the end date, set it and the occupation matrix
    '''
    orderedTasks = OrderTasksByPriorityAndPredecessor(tasks)
    for currtask in orderedTasks.itertuples():
        #print 'The current task is : %s' %(currtask.Index)        
        previousEndDate=_ProjectStartDate-_1DayTimeDelta
        predecessorTaskEndDate=_ProjectStartDate-_1DayTimeDelta
        
        #When did the last task finished
        previousTasks = FindPreviousTaskOfSomeone(tasks, currtask)  
        if(type(previousTasks)=='pandas.core.series.Series'):
            previousEndDate=previousTasks.EndDate
            if pd.isnull(previousEndDate):
                raise Exception('Couldn''t find the end date of the previous task',currtask.TaskId)

        #When did the predecessor finished if there is one
        if(pd.isnull(currtask.PredecessorTaskId)==False):            
            predecessorTask = FindTaskById(orderedTasks, currtask.PredecessorTaskId)
            #print '   %s %s' %(predecessorTask.TaskId, predecessorTask.EndDate)
            predecessorTaskEndDate=predecessorTask.EndDate
            if pd.isnull(predecessorTaskEndDate):
                raise Exception('Couldn''t find the end date of the predecessor task',currtask.TaskId)
        latestDate = max(predecessorTaskEndDate,previousEndDate)
        
        #The task start the next day of the previous
        orderedTasks.loc[currtask.Index,'StartDate']=latestDate+_1DayTimeDelta
        orderedTasks.loc[currtask.Index,'EndDate'] = ComputeEndDate(avail,orderedTasks.loc[currtask.Index])
        '''print 'Dates from %s to %s '%(orderedTasks.loc[currtask.Index,'StartDate'],\
                                      orderedTasks.loc[currtask.Index,'EndDate'])'''
    return orderedTasks
        

    

In [12]:
def FillAllOccupationFromTasks(tasks):
    occupation = InitEmptyOccupation(_avail,_ProjectStartDate,_ProjectStartDate+dt.timedelta(365))
    for task in tasks.itertuples():
        SetOccupiedBy(occupation,task)
    return occupation

#### Presentation

In [13]:
def GetPandaForGantChart(tasks):
    pdGant = tasks[['Name','Who','StartDate','EndDate']]
    pdGant.columns = ['Task','Resource','Start','Finish']    
    pdGant = pdGant.reset_index()
    del pdGant['TaskId']
    return pdGant

In [158]:
def DrawGant(gant):
    table = """[{ type: 'string', id: 'Who' },
        { type: 'string', id: 'TaskName' },
        { type: 'date', id: 'Start' },
        { type: 'date', id: 'End' }],
        """
    
    def PrintDate(date):
        return 'new Date(%s,%s,%s)'%(date.year,date.month-1,date.day)
    for gt in gant.itertuples():
        table = table + """['{}','{}',{},{}],
        """.format(gt.Resource,gt.Task.replace("'", "\\'"),PrintDate(gt.Start),PrintDate(gt.Finish+_1DayTimeDelta)) 
        #The end date is one day later otherwise it stops at 0:00 leaving a day gap
    #print 'google.visualization.arrayToDataTable([{}])'.format(table)
    js = """google.charts.load("current", {{packages:["timeline"]}});
        google.charts.setOnLoadCallback(drawChart);
         function drawChart() {{
            var container = document.getElementById('timeline');
            var chart = new google.visualization.Timeline(container);
            var options = {{
              timeline: {{ }}
         }};

         chart.draw(google.visualization.arrayToDataTable([{}]), options);
         }};
        """.format(table)
    #print js
    return dis.Javascript(js)

## Load Data

In [117]:
_tasks = LoadTask(_pathTasks)

In [113]:
_avail = LoadAvailability(_pathAvail)

In [118]:
_tasks.head(10)

,TaskId,Subject,Name,WorkLoad,Who,StartDate,EndDate,Priority,PredecessorTaskId
TaskId,,,,,,,,,
12,12,Telnet taux technique,Dev + UAT,3.0,ACH,NaT,NaT,1100,NaN
4,4,Reserves corporates,Injecter les donnees en DEV,1.0,PTI,NaT,NaT,1095,NaN
14,14,Chargements initiaux,Agresso,3.0,QPI,NaT,NaT,1090,NaN
31,31,BAU,Injection de la cloture,1.0,PTI,NaT,NaT,1087,NaN
15,15,Chargements initiaux,Vie,15.0,PTI,NaT,NaT,1085,NaN
5,5,Reserves corporates,Developper les flux datamarts,1.0,DEL,NaT,NaT,1075,4
39,39,Scoring,Preparation et Deploiement en UAT,2.0,CGA,NaT,NaT,1070,NaN
40,40,Scoring,Chargement datamart en UAT,2.0,CGA,NaT,NaT,1060,39
41,41,Scoring,Creation univers scoring,1.0,EGA,NaT,NaT,1050,NaN


In [114]:
_avail.head(5)

,Who,Date,Available
0,ACH,2016-11-21,1.0
2020,SLA,2016-11-21,1.0
3636,PM,2016-11-21,0.0
1616,MWA,2016-11-21,1.0
4040,BAU,2016-11-21,1.0


In [119]:
_tasks = ComputeOccupation(_avail,_tasks)

In [120]:
_tasks.head(10)

,TaskId,Subject,Name,WorkLoad,Who,StartDate,EndDate,Priority,PredecessorTaskId
TaskId,,,,,,,,,
12,12,Telnet taux technique,Dev + UAT,3.0,ACH,2016-11-21,2016-11-24,1100,NaN
4,4,Reserves corporates,Injecter les donnees en DEV,1.0,PTI,2016-11-21,2016-11-21,1095,NaN
14,14,Chargements initiaux,Agresso,3.0,QPI,2016-11-21,2016-11-28,1090,NaN
31,31,BAU,Injection de la cloture,1.0,PTI,2016-11-21,2016-11-21,1087,NaN
15,15,Chargements initiaux,Vie,15.0,PTI,2016-11-21,2016-12-14,1085,NaN
5,5,Reserves corporates,Developper les flux datamarts,1.0,DEL,2016-11-22,2016-11-22,1075,4
39,39,Scoring,Preparation et Deploiement en UAT,2.0,CGA,2016-11-21,2016-11-28,1070,NaN
40,40,Scoring,Chargement datamart en UAT,2.0,CGA,2016-11-29,2016-12-07,1060,39
41,41,Scoring,Creation univers scoring,1.0,EGA,2016-11-21,2016-11-21,1050,NaN


In [125]:
_occupation = FillAllOccupationFromTasks(_tasks)

      Who       Date  Available
0     ACH 2016-11-21        1.0
2020  SLA 2016-11-21        1.0
3636   PM 2016-11-21        0.0
1616  MWA 2016-11-21        1.0
4040  BAU 2016-11-21        1.0
2828  AHI 2016-11-21        1.0
1212  QPI 2016-11-21        1.0
4444  DEL 2016-11-21        1.0
808   PTI 2016-11-21        1.0
2424  OGO 2016-11-21        1.0
4848  EGA 2016-11-21        1.0
404   CGA 2016-11-21        1.0
5252  OVA 2016-11-21        1.0
3232  LZI 2016-11-21        1.0
2829  AHI 2016-11-22        1.0
1     ACH 2016-11-22        1.0
2021  SLA 2016-11-22        1.0
4849  EGA 2016-11-22        1.0
809   PTI 2016-11-22        1.0
4041  BAU 2016-11-22        1.0
1617  MWA 2016-11-22        1.0
1213  QPI 2016-11-22        0.0
4445  DEL 2016-11-22        1.0
2425  OGO 2016-11-22        1.0
405   CGA 2016-11-22        0.0
3637   PM 2016-11-22        1.0
5253  OVA 2016-11-22        1.0
3233  LZI 2016-11-22        1.0
810   PTI 2016-11-23        1.0
1618  MWA 2016-11-23        0.0
...   ..

In [144]:
_occupation.head(10)

,ACH,SLA,PM,MWA,BAU,AHI,QPI,DEL,PTI,OGO,EGA,CGA,OVA,LZI
2016-11-21,18,0,1,50,16,0,47,0,38,0,37,57,33,0
2016-11-22,18,0,1,50,16,0,47,3,38,0,37,57,0,0
2016-11-23,18,0,1,50,16,0,47,3,38,0,37,57,0,10
2016-11-24,18,20,1,50,16,23,47,0,38,24,37,57,0,10
2016-11-25,30,20,1,50,16,23,47,0,38,24,37,57,0,10
2016-11-26,30,20,1,50,16,23,47,0,38,24,21,57,0,10
2016-11-27,30,20,1,50,16,23,47,0,38,24,21,57,0,10
2016-11-28,30,20,1,50,16,23,47,0,38,24,21,57,0,10
2016-11-29,30,20,1,0,16,0,47,0,38,0,21,57,0,10
2016-11-30,30,20,1,0,16,0,47,0,38,0,21,57,0,10


In [145]:
_gant = GetPandaForGantChart(_tasks)

In [147]:
%%html
<script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>

In [156]:
%%html

       <div id="timeline" style="height: 250px;width: 3000px"></div>
   

In [157]:
DrawGant(_gant)

google.charts.load("current", {packages:["timeline"]});
        google.charts.setOnLoadCallback(drawChart);
         function drawChart() {
            var container = document.getElementById('timeline');
            var chart = new google.visualization.Timeline(container);
            var options = {
              timeline: { }
         };

         chart.draw(google.visualization.arrayToDataTable([[{ type: 'string', id: 'Who' },
        { type: 'string', id: 'TaskName' },
        { type: 'date', id: 'Start' },
        { type: 'date', id: 'End' }],
        ['ACH','Dev + UAT',new Date(2016,10,21),new Date(2016,10,25)],
        ['PTI','Injecter les donnees en DEV',new Date(2016,10,21),new Date(2016,10,22)],
        ['QPI','Agresso',new Date(2016,10,21),new Date(2016,10,29)],
        ['PTI','Injection de la cloture',new Date(2016,10,21),new Date(2016,10,22)],
        ['PTI','Vie',new Date(2016,10,21),new Date(2016,11,15)],
        ['DEL','Developper les flux datamarts',new Date(2016,10,2

<IPython.core.display.Javascript object>

# Todo
* Create a new html page 
* Better tooltip (html ?)